# SLOWLY ROTATING CASE IN ENTANGLED RELATIVITY FOR MAGNETICALLY CHARGED BLACK HOLES

For the details of the field equations in the theory of entangled relativity, we invite 
the reader to refer to the notebook ER_SR_Electric_BH describing the case of an electrically charged black hole in slow rotation. For the work of J. H. Horne and T. Horowitz (1992) - Physical Review D which provides solutions for arbitrary coupling of the dilaton with the Maxwell field, see the notebook SLOWLY ROTATING DILATON BLACK HOLES. 

We can deduce the magnetically charged solutions from the electric ones by making the following changes (see e.g. the article C.F.E. Holzhey & F. Wilczek (1992)- Nuclear Physics B):

$\Psi\rightarrow\Psi'=\frac{1}{\Psi}=\left[e^{2\alpha\phi}\right]_{|_{\alpha=(2\sqrt{3})^{-1}}}=\left[\left(1-\frac{r_{-}}{r}\right)^{\frac{-2\alpha^{2}}{1+\alpha^{2}}}\right]_{|_{\alpha=(2\sqrt{3})^{-1}}}=\left(1-\frac{r_{-}}{r}\right)^{\frac{2}{13}}$

and

$F_{\lambda\rho}^{el}\rightarrow F^{mag}_{\mu\nu}=e^{-2\alpha\phi}\frac{1}{2}\epsilon^{\lambda\rho}_{\mu\nu}F_{\lambda\rho}^{el}$ where $F_{\lambda\rho}^{el}$ is the electrically charged solution

The complexity of some calculations in this notebook prompted us to perform intermediate first order Taylor expansions to maintain tractability of the different operations using the Sagemath language. This does not affect the validity of our results since they are part of a first-order analysis in 'a'.

Additionally, it was necessary for us to take a very general approach here and to start from the Maxwell-Dilaton solution before applying an inverse conformal transformation to obtain the solution in ER. This allows us to replace the analytical expressions at the last moment and avoid Sage getting lost in the resolution.

In [1]:
version()

'SageMath version 10.1, Release Date: 2023-08-20'

In [2]:
%display latex

'SageMath version used is 10.1, Release Date: 2023-08-20'

In [3]:
from sage.manifolds.operators import dalembertian
from sage.manifolds.operators import grad

Let's write a function to replace all the different expressions and perform the taylor expansions 

In [4]:
def slow_rot(arg):
    subs_funcs = [(Z, E), (U, A), (V, B), (X, C), (Y, D), (O, Omega)]
    final_subs = {Q: H, alpha:1/(2*(3)**(1/2))}
    
    if hasattr(arg, 'expr'):
        arg = arg.expr()
        
    if hasattr(arg, 'apply_map')*hasattr(arg, 'display'):
        arg.apply_map(lambda f: f.taylor(a, 0, 1))
        for i, (old_func, new_func) in enumerate(subs_funcs):
            arg.apply_map(lambda f: (f.substitute_function(old_func, new_func).taylor(a, 0, 1) if i == 0 \
                                           else f.substitute_function(old_func, new_func)))                        
        arg.apply_map(lambda f: f.subs(final_subs).factor())
    else:
        arg = arg.taylor(a, 0, 1)
        for i, (old_func, new_func) in enumerate(subs_funcs):
            arg = (arg.substitute_function(old_func, new_func).taylor(a, 0, 1) if i == 0 \
                                           else arg.substitute_function(old_func, new_func)) 
        arg = arg.subs(final_subs).factor()()
    
    return arg

# I. Definition of the metric in ER

In [5]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

4-dimensional Lorentzian manifold M


In [6]:
XY.<t,r,th,ph> = M.chart(r't r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi')
XY

Chart (M, (t, r, th, ph))

In [7]:
g = M.metric()
r_m, r_p, alpha, a, Q = var('r_m r_p alpha a Q')

In [8]:
U = function('U')
V = function('V')
X = function('X')
Y = function('Y')
Z = function('Z')
O = function('O')
g[0,0] = U(r)
g[0,3] = Z(r,th) 
g[1,1] = V(r)
g[2,2] = X(r)
g[3,3] = Y(r,th)
g[3,0] = Z(r,th) 

In [9]:
Psi(r) = (1-r_p/r)
Phi(r) = (1-r_m/r)
A(r)=-Psi(r)*Phi(r)**((1-alpha**2)/(1+alpha**2))
B(r)=-A(r)**(-1)
C(r)=r**2*Phi(r)**(2*alpha**2/(1+alpha**2))
D(r,th)=sin(th)**2*C(r)
f(r)=(r**2*(1+alpha**2)**2)*Phi(r)**(2*alpha**2/(1+alpha**2))/(((1-alpha**2)*(1-3*alpha**2)*r_m**2))-Phi(r)**((1-alpha**2)/(1+alpha**2))*(1+(1+alpha**2)**2*r**2/((1-alpha**2)*(1-3*alpha**2)*r_m**2)+(1+alpha**2)*r/((1-alpha**2)*r_m)-r_p/r)
E(r,th)=-a*sin(th)**2*f(r)
H=(r_m*r_p/(1+alpha**2))**(1/2)

## I.1. The scalar field

In [10]:
Phi_sc = M.scalar_field((1-r_m/r)**(2*alpha**2/(1+alpha**2)), name='Phi')
Phi_sc.display()

Phi: M → ℝ
   (t, r, th, ph) ↦ (-r_m/r + 1)^(2*alpha^2/(alpha^2 + 1))

## I.2. Conformal transformation

In [11]:
g.apply_map(lambda f: f*O(r)**2) ; g.display()

g = O(r)^2*U(r) dt⊗dt + O(r)^2*Z(r, th) dt⊗dph + O(r)^2*V(r) dr⊗dr + O(r)^2*X(r) dth⊗dth + O(r)^2*Z(r, th) dph⊗dt + O(r)^2*Y(r, th) dph⊗dph

In [12]:
Omega(r)=1/(Phi_sc.expr())  ;  Omega(r).subs(alpha=1/(2*(3)**(1/2)))

(-r_m/r + 1)^(-2/13)

In [13]:
h = g.copy()
h.set_name('h')
h.display()

h = O(r)^2*U(r) dt⊗dt + O(r)^2*Z(r, th) dt⊗dph + O(r)^2*V(r) dr⊗dr + O(r)^2*X(r) dth⊗dth + O(r)^2*Z(r, th) dph⊗dt + O(r)^2*Y(r, th) dph⊗dph

We substitute the different functions U,V X,Y,Z,Q and Omega in the expression 

In [14]:
slow_rot(h).display()

h = -(r - r_p)*((r - r_m)/r)^(7/13)/r dt⊗dt - 1/99*(169*r^4*((r - r_m)/r)^(4/13) - 169*r^4 + 52*r^3*r_m + 18*r^2*r_m^2 + 99*r*r_m^3 + 99*r*r_m^2*r_p - 99*r_m^3*r_p)*a*((r - r_m)/r)^(7/13)*sin(th)^2/((r - r_m)*r*r_m^2) dt⊗dph + r/((r - r_p)*((r - r_m)/r)^(15/13)) dr⊗dr + r^2/((r - r_m)/r)^(2/13) dth⊗dth - 1/99*(169*r^4*((r - r_m)/r)^(4/13) - 169*r^4 + 52*r^3*r_m + 18*r^2*r_m^2 + 99*r*r_m^3 + 99*r*r_m^2*r_p - 99*r_m^3*r_p)*a*((r - r_m)/r)^(7/13)*sin(th)^2/((r - r_m)*r*r_m^2) dph⊗dt + r^2*sin(th)^2/((r - r_m)/r)^(2/13) dph⊗dph

In [15]:
slow_rot((h[0,3].expr())).simplify()

-1/99*(169*r^4*((r - r_m)/r)^(4/13) - 169*r^4 + 52*r^3*r_m + 18*r^2*r_m^2 + 99*r*r_m^3 + 99*r*r_m^2*r_p - 99*r_m^3*r_p)*a*((r - r_m)/r)^(7/13)*sin(th)^2/((r - r_m)*r*r_m^2)

# II. The vector potential

Let's define the covariant derivative

In [16]:
nab = g.connection() 

and the potential vector field corresponding to a slowly rotating BH

In [17]:
pot_vec = M.tensor_field(0,1,name='A')
Q=var('Q')

A_t = -a*Q*13*(13-(1-r_m/r)**(9/13)*(13+9*r_m/r))/(99*r_m**2)*sin(th)
A_phi = -Q*sin(th)

pot_vec[0]= -integrate(A_t,th).factor()
pot_vec[1]=0
pot_vec[2]=0
pot_vec[3]=integrate(A_phi,th).factor()
pot_vec.display()

A = 13/99*(13*r*((r - r_m)/r)^(9/13) + 9*r_m*((r - r_m)/r)^(9/13) - 13*r)*Q*a*cos(th)/(r*r_m^2) dt + Q*cos(th) dph

## II.1. Definition of the EM tensor

Let's now define the EM tensor:

$F_{\mu\nu}=\partial_{\mu}A_{\nu}-\partial_{\nu}A_{\mu}$

It can be obtained directly as the exterior derivative of the potential vector $A$ via the function `diff`
(or the method `exterior_derivative`)

In [18]:
F = diff(pot_vec)
F.set_name('F')
Fuu = F.up(g)
F.display()

F = -2*Q*a*cos(th)/((r - r_m)^(4/13)*r^(35/13)) dt∧dr - 13/99*(13*Q*a*r^(22/13)*sin(th) - (13*Q*a*r + 9*Q*a*r_m)*(r - r_m)^(9/13)*sin(th))/(r^(22/13)*r_m^2) dt∧dth - Q*sin(th) dth∧dph

In [19]:
Fm = F
Fm.apply_map(lambda f:f.substitute_function(Z,E).subs(alpha=1/(2*(3)**(1/2))).taylor(a,0,1))

In [20]:
Fm_c = Fm.copy()
Fm_c.apply_map(lambda f:slow_rot(f).subs(alpha=1/(2*sqrt(3))).canonicalize_radical())
show(LatexExpr(r'\mathcal{F}^{m}_{\mu\nu} = '), Fm_c.display())

\mathcal{F}^{m}_{\mu\nu} =  -4/13*sqrt(13)*sqrt(3)*a*sqrt(r_m)*sqrt(r_p)*cos(th)/((r - r_m)^(4/13)*r^(35/13)) dt∧dr - 2/99*sqrt(3)*(13*sqrt(13)*a*r^(22/13)*sqrt(r_p)*sin(th) - (13*sqrt(13)*a*r + 9*sqrt(13)*a*r_m)*(r - r_m)^(9/13)*sqrt(r_p)*sin(th))/(r^(22/13)*r_m^(3/2)) dt∧dth - 2/13*sqrt(13)*sqrt(3)*sqrt(r_m)*sqrt(r_p)*sin(th) dth∧dph

In [21]:
Lm = -Fm['_ij']*Fm.up(g)['^ij']/2
Lm = slow_rot(Lm)
show(LatexExpr(r'\mathcal{L}_{m} = '), Lm, LatexExpr(r'+ \mathcal{O}(a^{2})'))

\mathcal{L}_{m} =  -r_m*r_p*((r - r_m)/r)^(4/13)/((alpha^2 + 1)*r^4) + \mathcal{O}(a^{2})

# III. The motion equation

## III.1 Left hand side of the motion equation (1) : the Einstein tensor

In [22]:
g.display()

g = O(r)^2*U(r) dt⊗dt + O(r)^2*Z(r, th) dt⊗dph + O(r)^2*V(r) dr⊗dr + O(r)^2*X(r) dth⊗dth + O(r)^2*Z(r, th) dph⊗dt + O(r)^2*Y(r, th) dph⊗dph

In [23]:
ER_ricc = g.ricci()
slow_rot(ER_ricc)

Field of symmetric bilinear forms Ric(g) on the 4-dimensional Lorentzian manifold M

In [24]:
gu = g.up(g) 
slow_rot(gu)

Tensor field of type (2,0) on the 4-dimensional Lorentzian manifold M

In [25]:
ER_rsc = gu['^ab']*ER_ricc['_ab']
show(LatexExpr(r'R = '), slow_rot(ER_rsc), LatexExpr(r'+ \mathcal{O}(a^{2})'))

R =  12/13*(r - r_m)^(2/13)*r_m*r_p/r^(54/13) + \mathcal{O}(a^{2})

We contruct the Einstein tensor

In [26]:
G = ER_ricc - 1/2*g*ER_rsc
G.set_name(r'G_{\mu\nu}')
slow_rot(G)

Field of symmetric bilinear forms G_{\mu\nu} on the 4-dimensional Lorentzian manifold M

## III.2. The right hand side of the motion equation (1)

### III.2.1. The first component: with the stress-energy tensor $T_{\mu\nu}=2\left(F_{\rho\mu}F^{\rho}_{\hspace{0.2cm}\nu}-\frac{1}{4}g_{\mu\nu}F^{2}\right)$ 

In [27]:
Fuu = Fm.up(g)
F2 = Fm['_ab']*Fuu['^ab']
F2 = F2.expr().substitute_function(Z,E).taylor(a,0,1)

In [28]:
Fud = Fm.up(g,0)
T = 2*(Fm['_k.']*Fud['^k_.'] - 1/4*F2 * g)
slow_rot(T)

Tensor field of type (0,2) on the 4-dimensional Lorentzian manifold M

We deduce the final quotient and perform a first first-order Taylor expansion to simplify the expression before any further calculations

In [29]:
UU=T/Phi_sc
slow_rot(UU)

Tensor field of type (0,2) on the 4-dimensional Lorentzian manifold M

### III.2.3 The second component of the motion equation $\frac{1}{\Phi}(\triangledown_{\mu}\triangledown_{\nu}-g_{\mu\nu}\square)\Phi$

We now construct the d'Alembert operator

In [30]:
Dal = dalembertian(Phi_sc**2); Dal

Scalar field Box(Phi^2) on the 4-dimensional Lorentzian manifold M

In [31]:
Gp = grad(Phi_sc**2); Gp 

Vector field grad(Phi^2) on the 4-dimensional Lorentzian manifold M

We substitute the function Z in the expression and we perform a first first-order Taylor expansion to simplify the expression before any further calculations

In [32]:
S = Phi_sc**(-2)*(nab(Gp).down(g)-g*Dal) 
S.apply_map(lambda f: f.substitute_function(Z,E))
S.apply_map(lambda f: f.taylor(a,0,1)) 

We subsitute the remaining expressions U,V,X,Y,Q and O to have the final form of the right hand side of the motion equation (1)

In [33]:
RHS = UU + S
slow_rot(RHS)

Tensor field of type (0,2) on the 4-dimensional Lorentzian manifold M

# IV. Verification of the field equations

## IV.1. Verification of the $G_{\mu\nu} = \tilde{k} \frac{T_{\alpha\beta}}{\sqrt{\phi}}+\frac{1}{\phi}\left[\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right]\phi$ equation

Let's check that the two sides of the equation (1) are equal

In [34]:
G == RHS

True

## IV.2. Verification of the Maxwell equations $\triangledown\left(\sqrt{\phi}F_{\mu\nu}\right)$ 

In [35]:
slow_rot(Fuu)

Tensor field of type (2,0) on the 4-dimensional Lorentzian manifold M

In [36]:
q = Fm.up(g)*Phi_sc
slow_rot(q)

Tensor field of type (2,0) on the 4-dimensional Lorentzian manifold M

In [37]:
eq = nab(q)['_i^ij']
slow_rot(eq)

Vector field on the 4-dimensional Lorentzian manifold M

In [38]:
latex_str = r'\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{m} = ' 
show(LatexExpr(latex_str), eq[:])

\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{m} =  [0, 0, 0, 0]

In [50]:
assume(r > r_m)
assume(r > r_p)
expression = eq[0].expr().simplify().factor()
latex_str = r'\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{00} = ' 
show(LatexExpr(latex_str),expression)
expression.is_zero()

\triangledown\left(\sqrt{\phi}F^{\mu\nu}\right)_{00} =  0

True

Therefore, $\triangledown\left(\sqrt{\phi_{m}}F^{\mu\nu}\right) = 0$ is always true outside the black hole, i.e. for $r>r_{m}$ and $r>r_{p}$. 

As the solution does describe a black hole iff $r_{m}<r_{p}$, the condition $r>r_{p}$ is sufficient

In conclusion, Maxwell equations are verified.

## IV.3.  Verification of the $\vartheta = -\frac{R}{L_{m}}$ equation

In [49]:
show(LatexExpr(r'\Phi ='), slow_rot(Phi_sc))
show(LatexExpr(r'\frac{L_{m}}{R} ='), slow_rot(Lm / (slow_rot(ER_rsc))).canonicalize_radical().factor())
show(LatexExpr(r'\phi + \frac{Lm}{R} ='), (slow_rot(Phi_sc) + slow_rot(Lm / ER_rsc)).canonicalize_radical())
(slow_rot(Phi_sc) + slow_rot(Lm / ER_rsc)).canonicalize_radical().is_zero()

\Phi = ((r - r_m)/r)^(2/13)

\frac{L_{m}}{R} = -(r - r_m)^(2/13)/r^(2/13)

\phi + \frac{Lm}{R} = 0

True

# In conclusion the equation $G_{\mu\nu} = \tilde{k} \frac{T_{\alpha\beta}}{\sqrt{\phi}}+\frac{1}{\phi}\left[\nabla_{\mu}\nabla_{\nu} - g_{\mu\nu}\square\right]\phi$ is perfectly verified at the first order and the $\triangledown\left(\sqrt{\phi}T_{\mu\nu}\right)$ is always verified outside the black hole.

# So the magnetically charged solutions are solutions in ER as well for a slowly rotating black hole.